#User Interface for Personalized Discharge Summary

##Retrieve Data

In [90]:
import pandas as pd

# Read into a DataFrame
df = pd.read_csv('data.csv', nrows=55)

# Select patient to display
pt_num = 1

# Get all medications for patient
meds = df.iloc[pt_num, 6:50]
meds = meds.tolist()
meds = pd.Series(meds).dropna().tolist()

# Get diagnoses for patient
diags = [df.iloc[pt_num, 51]]

# Get discharge instructions for patient
disch_inst = df.iloc[pt_num, 74]

##Organize data and explanations for keywords to be displayed

In [91]:
import json

# Structure data
data = {
    "Diagnoses": diags,
    "Medications": meds,
    "Discharge Instructions": (
        disch_inst
    ),
    "ActionableItems":  json.load(open("final_actions.json"))
}
data_json = json.dumps(data)

# Keywords and their explanations
keywords_json = json.load(open("finalized_output.json"))

##HTML, CSS, Javascript for UI

In [92]:
from IPython.display import display, HTML

html_content = f"""
<div class="container">
    <h1>🏥 My Discharge Summary</h1>

    <!-- Font Size Control -->
    <div class="font-control">
        <label for="fontSize">Adjust font size:</label>
        <select id="fontSize">
            <option value="14px">Small</option>
            <option value="18px" selected>Medium</option>
            <option value="22px">Large</option>
            <option value="26px">Extra Large</option>
        </select>
    </div>

    <div class="card">
        <h2>🩺 Diagnoses</h2>
        <ul id="diagnoses-list"></ul>
    </div>

    <div class="card">
        <h2>💊 Medications</h2>
        <ul id="medications-list"></ul>
    </div>

    <div class="card">
        <h2>📋 Discharge Instructions</h2>
        <p id="instructions-text"></p>
    </div>

    <div class="card">
        <h2>✅ Actionable Items</h2>
        <ul id="actitems-list"></ul>
    </div>
</div>

<style>
.container {{
    font-family: 'Helvetica Neue', Arial, sans-serif;
    color: #1a1a1a;
    background: #f9fbfd;
    padding: 30px;
    border-radius: 16px;
    max-width: 900px;
    margin: 40px auto;
    box-shadow: 0 0 20px rgba(0,0,0,0.05);
    transition: font-size 0.2s ease;
}}
h1 {{
    color: #042a70;
    text-align: center;
    margin-bottom: 20px;
}}
.font-control {{
    text-align: center;
    margin-bottom: 20px;
}}
select {{
    margin-left: 10px;
    padding: 5px;
    border-radius: 6px;
    border: 1px solid #ccc;
}}
h2 {{
    color: #075099;
    border-bottom: 2px solid #004d99;
    padding-bottom: 5px;
    margin-bottom: 15px;
}}
ul {{
    list-style-type: none;
    padding-left: 0;
}}
li {{
    padding: 6px 0;
    border-bottom: 1px solid #eee;
}}
.card {{
    background: white;
    border-radius: 12px;
    padding: 20px 25px;
    margin-bottom: 25px;
    box-shadow: 0 2px 10px rgba(0,0,0,0.05);
}}
.keyword {{
    font-weight: 600;
    color: #006e78;
    cursor: pointer;
    position: relative;
    transition: color 0.2s;
}}
.keyword:hover {{
    color: #006e78;
}}
.tooltip {{
    display: none;
    position: absolute;
    top: 22px;
    left: 0;
    background: #ffffff;
    border: 1px solid #d4ddee;
    border-radius: 8px;
    padding: 8px;
    z-index: 100;
    width: 280px;
    box-shadow: 0 2px 12px rgba(0,0,0,0.15);
    animation: fadeIn 0.2s ease-in-out;
}}
@keyframes fadeIn {{
    from {{opacity: 0; transform: translateY(-4px);}}
    to {{opacity: 1; transform: translateY(0);}}
}}
.tooltip button {{
    background: #e6f0ff;
    border: none;
    border-radius: 6px;
    padding: 4px 8px;
    margin: 2px;
    cursor: pointer;
    font-size: inherit;
    transition: background 0.2s;
}}
.tooltip button:hover {{
    background: #cce0ff;
}}
.tooltip p {{
    margin-top: 6px;
    font-size: inherit;
    color: #333;
}}
input[type="checkbox"] {{
    transform: scale(1.2);
    margin-right: 10px;
    cursor: pointer;
}}
</style>
<script>
const data = {data_json};
const keywords = {keywords_json};

function highlightKeywords(text) {{
    for (const key in keywords) {{
        const escapedKey = key.replace(/[.*+?^${{}}()|[\\]\\\\]/g, '\\\\$&');
        const regex = new RegExp(escapedKey, "gi");
        text = text.replace(regex, (match) => {{
            return `<span class="keyword">${{match}}
                        <div class="tooltip">
                            <div style='font-weight:600; margin-bottom:4px;'>Explain:</div>
                            <button onclick="this.closest('.tooltip').querySelector('p').innerText=keywords['${{key}}'].basic">Basic</button>
                            <button onclick="this.closest('.tooltip').querySelector('p').innerText=keywords['${{key}}'].intermediate">Intermediate</button>
                            <button onclick="this.closest('.tooltip').querySelector('p').innerText=keywords['${{key}}'].advanced">Advanced</button>
                            <p></p>
                        </div>
                    </span>`;
        }});
    }}
    return text;
}}

// Add hover delay to tooltips
function addTooltipDelay() {{
    const tooltips = document.querySelectorAll('.tooltip');
    tooltips.forEach(t => {{
        let timeout;
        const parent = t.parentElement;
        parent.addEventListener('mouseenter', () => {{
            clearTimeout(timeout);
            t.style.display = 'block';
        }});
        parent.addEventListener('mouseleave', () => {{
            timeout = setTimeout(() => {{ t.style.display = 'none'; }}, 300);
        }});
        t.addEventListener('mouseenter', () => {{
            clearTimeout(timeout);
        }});
        t.addEventListener('mouseleave', () => {{
            timeout = setTimeout(() => {{ t.style.display = 'none'; }}, 300);
        }});
    }});
}}

// Populate Diagnoses
const diagList = document.getElementById("diagnoses-list");
data.Diagnoses.forEach(d => {{
    const diag = document.createElement("li");
    diag.innerHTML = highlightKeywords(d);
    diagList.appendChild(diag);
}});

// Populate Medications
const medList = document.getElementById("medications-list");
data.Medications.forEach(m => {{
    const med = document.createElement("li");
    med.innerHTML = highlightKeywords(m);
    medList.appendChild(med);
}});

// Populate Discharge Instructions
document.getElementById("instructions-text").innerHTML = highlightKeywords(data["Discharge Instructions"]);

// Populate Actionable Items
const actItemsList = document.getElementById("actitems-list");
data.ActionableItems.forEach(a => {{
    const actItem = document.createElement("li");
    actItem.innerHTML = `<label><input type="checkbox"> ${{highlightKeywords(a)}}</label>`;
    actItemsList.appendChild(actItem);
}});

// Apply tooltip hover delay
addTooltipDelay();

// --- Font Size Control ---
document.getElementById("fontSize").addEventListener("change", function() {{
    document.querySelector('.container').style.fontSize = this.value;
}});
</script>
"""

display(HTML(html_content))
